<a href="https://colab.research.google.com/github/dgabriel0/Tabela-Brasileirao/blob/main/Campeonato_brasileiro_2012_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
for ano in range(2012,2024):
  url = f'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a/{ano}'
  reqs = requests.get(url)
  soup = BeautifulSoup(reqs.text, 'html.parser')
  #usei a classe icon escudo pq nela vem o nome completo, se usasse a classe sigla-escudo ela me retornaria obviamente a sigla, e geraria conflito, pois a sigla do atletico - mg(ATL) é igual a do atletico - go(ATL)
  timeA = []
  timeB = []
  timeA = soup.findAll('img',class_={'icon escudo x45 pull-right'})
  timeB = soup.findAll('img',class_={'icon escudo x45 pull-left'})
  #separando todos os confrontos do ano
  timeCasa = []
  timeFora = []

  for i in range(len(timeA)):

      timeCasa.append(timeA[i].get('title'))



      timeFora.append(timeB[i].get('title'))
  #organizando todos os 20 times que participão da serie A desse ano
  participantes = timeCasa[0:10] + timeFora[0:10]
  rodadas = []

  for i in range(1,39):
    for x in range(0,10):
      rodadas.append(i)
  resultado = []
  placar = soup.findAll('span',class_={'bg-blue color-white label-2'})
  for resul in placar:
    resultado.append(resul.get_text())
  #devido ao acidente da chapecoense em 2016, não houve a ultima partida entre galo e chape, estou colocando 0 x 1 mas irei tirar os pontos do Galo
  if ano == 2016:
    resultado.insert(-2,'0 x 0')
  #Depois de analisar o codigo fonte descobri que a partir de 2018
  #o site da cbf repete o html da rodada 38, no começo do html, fazendo o soup pegar 20 resultados a+
  if ano == 2018 or ano == 2019 or ano == 2020 or ano == 2021 or ano == 2022 or ano == 2023:
    resultado = resultado[20:]
  confrontos = pd.DataFrame()
  confrontos['Rodada'] = rodadas
  confrontos['Casa'] = timeCasa
  confrontos['Placar'] = resultado
  confrontos['Fora'] = timeFora
  golCasa = []
  golFora = []
  for i in range(len(confrontos)):
    golCasa.append(confrontos['Placar'][i].split(' x ')[0])
    golFora.append(confrontos['Placar'][i].split(' x ')[1])
  confrontos.insert(2, 'Gols Casa', golCasa)
  confrontos.insert(5, 'Gols Fora',golFora)

  del confrontos['Placar']
  confrontos
  #organizando o numero de pontos de cada time por partida
  casaPontos = []
  foraPontos = []
  try:
    for i in range(len(confrontos)):
        if int(sorted(confrontos['Gols Casa'][i].split(')'), reverse=True)[0]) == int(sorted(confrontos['Gols Fora'][i].split(')'), reverse = True)[0]):
          casaPontos.append(1)
          foraPontos.append(1)
          continue
        if int(sorted(confrontos['Gols Casa'][i].split(')'), reverse=True)[0]) > int(sorted(confrontos['Gols Fora'][i].split(')'), reverse = True)[0]):
          casaPontos.append(3)
          foraPontos.append(0)
          continue
        if int(sorted(confrontos['Gols Casa'][i].split(')'), reverse=True)[0]) < int(sorted(confrontos['Gols Fora'][i].split(')'), reverse = True)[0]):
          casaPontos.append(0)
          foraPontos.append(3)
          continue
  except:
    continue
  confrontos['Pontos Casa'] = casaPontos
  confrontos['Pontos Fora'] = foraPontos

  df = confrontos.copy()
  #criando um DataFrame exclusivo a classificação
  tabelaBrasileirao = pd.DataFrame(columns=['Rodada','Time', 'Pontos'])

  #esse loop verifica se o time jogou dentro ou fora de casa e pega os dados relevantes da primeira tabela
  count = 0
  rodadaAux = []
  timeAux = []
  pontosAux = []
  emCasa = []
  golsProAux = []
  golsContraAux = []
  golsPro = 0
  golsContra = 0
  vitorias = 0
  vitoriasAux = []
  derrotas = 0
  derrotasAux = []
  empates = 0
  empatesAux = []
  for x in range(0,20):
    golsPro = 0
    golsContra = 0
    empates = 0
    derrotas = 0
    vitorias = 0
    for i in range(1,39):
      if not(df.loc[((df['Rodada'] == i) & (df['Casa'] == participantes[x]))].empty):

        rodadaAux.append(i)

        timeAux.append(participantes[x])

        pontosAux.append(list(df['Pontos Casa'].loc[((df['Rodada'] == i) & (df['Casa'] == participantes[x]))])[0])

        emCasa.append('Casa')

        golsPro += int(sorted(list(df['Gols Casa'].loc[((df['Rodada'] == i) & (df['Casa'] == participantes[x]))])[0].split(')'),reverse = True)[0])
        golsProAux.append(golsPro)

        golsContra += int(sorted(list(df.loc[((df['Rodada'] == i) & (df['Casa'] == participantes[x]))]['Gols Fora'])[0].split(')'),reverse = True)[0])
        golsContraAux.append(golsContra)

        if list(df['Pontos Casa'].loc[((df['Rodada'] == i) & (df['Casa'] == participantes[x]))])[0] == 0:
          derrotas +=1
          derrotasAux.append(derrotas)
          empatesAux.append(empates)
          vitoriasAux.append(vitorias)
        if list(df['Pontos Casa'].loc[((df['Rodada'] == i) & (df['Casa'] == participantes[x]))])[0] == 1:
          empates +=1
          empatesAux.append(empates)
          vitoriasAux.append(vitorias)
          derrotasAux.append(derrotas)
        if list(df['Pontos Casa'].loc[((df['Rodada'] == i) & (df['Casa'] == participantes[x]))])[0] == 3:
          vitorias +=1
          vitoriasAux.append(vitorias)
          derrotasAux.append(derrotas)
          empatesAux.append(empates)


      if not(df.loc[((df['Rodada'] == i) & (df['Fora'] == participantes[x]))].empty):

        rodadaAux.append(i)
        timeAux.append(participantes[x])
        pontosAux.append(list(df['Pontos Fora'].loc[((df['Rodada'] == i) & (df['Fora'] == participantes[x]))])[0])
        emCasa.append('Fora')
        golsPro += int(sorted(list(df['Gols Fora'].loc[((df['Rodada'] == i) & (df['Fora'] == participantes[x]))])[0].split(')'),reverse = True)[0])
        golsProAux.append(golsPro)
        golsContra += int(sorted(list(df.loc[((df['Rodada'] == i) & (df['Fora'] == participantes[x]))]['Gols Casa'])[0].split(')'),reverse = True)[0])
        golsContraAux.append(golsContra)
        if list(df['Pontos Fora'].loc[((df['Rodada'] == i) & (df['Fora'] == participantes[x]))])[0] == 0:
          derrotas +=1
          derrotasAux.append(derrotas)
          empatesAux.append(empates)
          vitoriasAux.append(vitorias)
        if list(df['Pontos Fora'].loc[((df['Rodada'] == i) & (df['Fora'] == participantes[x]))])[0] == 1:
          empates +=1
          empatesAux.append(empates)
          vitoriasAux.append(vitorias)
          derrotasAux.append(derrotas)
        if list(df['Pontos Fora'].loc[((df['Rodada'] == i) & (df['Fora'] == participantes[x]))])[0] == 3:
          vitorias +=1
          vitoriasAux.append(vitorias)
          derrotasAux.append(derrotas)
          empatesAux.append(empates)
  saldoGols = []

  for i in range(len(golsProAux)):
    saldoGols.append(golsProAux[i] - golsContraAux[i])

  tabelaBrasileirao['Rodada'] = rodadaAux
  tabelaBrasileirao['Time'] =timeAux
  tabelaBrasileirao['Vitorias'] = vitoriasAux
  tabelaBrasileirao['Empates'] = empatesAux
  tabelaBrasileirao['Derrotas'] = derrotasAux
  tabelaBrasileirao['Pontos'] = pontosAux
  tabelaBrasileirao['Em Casa?'] = emCasa
  tabelaBrasileirao['Total de Gols'] = golsProAux
  tabelaBrasileirao['Total de Gols Tomados'] = golsContraAux
  tabelaBrasileirao['Saldo de Gols'] = saldoGols
  tabelaBrasileirao = tabelaBrasileirao.sort_values(['Time','Rodada'], ignore_index=True)

  count = 0
  pontosCum = []
  pontos = 0
  saldoDeGols = []
  gols = 0
  try:
    for i in range(0,20):

      rangemax = count+38
      for x in range(count,rangemax):
        pontos +=tabelaBrasileirao['Pontos'][x]
        pontosCum.append(pontos)

      count += 38
      pontos = 0
  except:
      print('except')

  tabelaBrasileirao['Pontos Acumulados']=pontosCum
  tabelaBrasileirao['Aproveitamento %'] = round((tabelaBrasileirao['Pontos Acumulados'] /( (tabelaBrasileirao['Rodada'])*3))*100,2)
  #devido ao acidente da chapecoense em 2016
  if ano == 2016:

    tabelaBrasileirao.at[(tabelaBrasileirao.loc[((tabelaBrasileirao['Time'] == 'Atlético - MG') & (tabelaBrasileirao['Rodada'] == 38))].index)[0],'Pontos'] = 62
    tabelaBrasileirao.at[(tabelaBrasileirao.loc[((tabelaBrasileirao['Time'] == 'Atlético - MG') & (tabelaBrasileirao['Rodada'] == 38))].index)[0],'Empates'] = 11
    tabelaBrasileirao.at[(tabelaBrasileirao.loc[((tabelaBrasileirao['Time'] == 'Chapecoense - SC') & (tabelaBrasileirao['Rodada'] == 38))].index)[0],'Pontos'] = 52
    tabelaBrasileirao.at[(tabelaBrasileirao.loc[((tabelaBrasileirao['Time'] == 'Chapecoense - SC') & (tabelaBrasileirao['Rodada'] == 38))].index)[0],'Empates'] = 13

  tabelaBrasileirao.to_csv(f'Tabela {ano}.csv')
  tabelaBrasileirao.to_excel(f'Tabela {ano}.xlsx')
  df.to_csv(f'Partidas {ano}.csv')
  df.to_excel(f'Partidas {ano}.xlsx')











In [61]:

# por algum motivo um valor esta retornando assim: (-)0 e se estiver dentro de int() ele retorna um erro
#pra resolver isso basta dar um split, mas isso só funciona nesse caso, nos outros valores normais retorna erro de index
#pra corrigir usei o sorted, com o atributo reverse, pra sempre deixar o numero na posição 0 e não na posição 1

strs = '0'
sasda = ')0'


print(int((sorted(strs.split(')'), reverse = True)[0])))


0


In [62]:
df.at[(df.loc[((df['Casa'] == 'Palmeiras - SP') & (df['Rodada'] == 1))].index)[0],'Pontos Casa']

3